In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm_notebook, tqdm
import itertools
from sklearn.impute import KNNImputer
from sklearn import preprocessing
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
from imblearn.over_sampling import BorderlineSMOTE
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
# функция удаления нулевых столбцов
def del_zero (data):
    zero_value=[]
    for i in range(1,261):
        if data.iloc[:,i].sum()==0: zero_value.append(data.iloc[:,i].name)
    print('Пустые столбцы ',zero_value, ' удалены')
    data=data.drop(zero_value, axis=1)
    return data
    
# функция удаления  признаков с пропущенными значениями число которых больше порога a_del
def del_na(data, a_del):
    na_values=pd.DataFrame(columns=[['null_sum','null_in_0','null_in_1', 'null_in_2']])
    for i in range(0,data.shape[1]):
        temp=[]
        if data.iloc[:,i].isnull().sum()>a_del:
            temp.append(data.iloc[:,i].isnull().sum())
            temp.append(data.query('TARGET==0').iloc[:,i].isnull().sum())
            temp.append(data.query('TARGET==1').iloc[:,i].isnull().sum())
            temp.append(data.query('TARGET==2').iloc[:,i].isnull().sum())
            nm=data.iloc[:,i].name
            na_values.loc[nm]=temp
    data=data.drop(list(na_values.index), axis=1)
    print(' Признаки с количеством порпусков >',a_del, '\n', na_values, ' удалены')
    return(data)

# удалим признаки в которых есть сильный дисбаланс, одно из значений подавляющее (больше порога max_count)
# из далее построенных гистограм, был сделан вывод что интересным может показаться порог отсечения _______
def del_disbal(data, max_count):
    disbal_values=pd.DataFrame(columns=[['max_sum','max_in_0','max_in_1','max_in_2']])
    for i in range(0,data.shape[1]):
        temp=[]
        if data.iloc[:,i].value_counts().max()>max_count:
            temp.append(data.iloc[:,i].value_counts().max())
            temp.append(data.query('TARGET==0').iloc[:,i].value_counts().max())
            temp.append(data.query('TARGET==1').iloc[:,i].value_counts().max())
            temp.append(data.query('TARGET==2').iloc[:,i].value_counts().max()) 
            nm=data.iloc[:,i].name
            disbal_values.loc[nm]=temp
    data=data.drop(list(disbal_values.index), axis=1)
    print(' Признаки с дисбалансом >',max_count, '\n', disbal_values, ' удалены')
    return(data)

def del_high_corr(data, cut_off=0.7):
    
    """
    Функция предназначена для удаления признаков, у которых корреляция выше некоторого порога.
    Функция принимает следующие параметры:
    data: pandas.DataFrame - данные, для которых будет рассчитана матрица корреляций;
    cut_off: float - порог отсечения коррелирующих признаков, значение по умолчанию равно 0.7,
    что в соответствии с таблицей Чеддока соответствует высокой силе корреляционной связи.
    Функция ничего не возвращает, она удаляет признаки, корреляция которых превышает установленный
    порог.
    """
    
    i, to_drop = 0, []
    while True:
        corr_ = data.corr(method='spearman') >= cut_off
        tmp = list(zip(corr_.sum().index, corr_.sum()))
        if tmp[i][1] > 1:
            data.drop(tmp[i][0], axis=1, inplace=True)
            print(f'Признак {tmp[i][0]} был удалён')
        else:
            i +=1
        if data.shape[1] == corr_.sum().values.sum():
            break

# опишем функцию заполнения пропущенных значений
def imputer(data, n_n):
    imp = KNNImputer(n_neighbors=n_n, weights="uniform")
    not_nan_mass=imp.fit_transform(data)
    ft_list=list(data.columns)
    data_not_nan=pd.DataFrame(not_nan_mass, columns=ft_list)
    return(data_not_nan)

# ВСПОМОГАТЕЛЬНАЯ ФУНКЦИЯ посмотрим на признаки с пропущенными значениями и их представленность в каждом классе
def na_val(data):
    na_values=pd.DataFrame(columns=[['null_sum','null_in_0','null_in_1', 'null_in_2']])
    for i in range(0,data.shape[1]):
        temp=[]
        if data.iloc[:,i].isnull().sum()>0:
            temp.append(data.iloc[:,i].isnull().sum())
            temp.append(data.query('TARGET==0').iloc[:,i].isnull().sum())
            temp.append(data.query('TARGET==1').iloc[:,i].isnull().sum())
            temp.append(data.query('TARGET==2').iloc[:,i].isnull().sum())
            nm=data.iloc[:,i].name
            na_values.loc[nm]=temp
    return(na_values)
    
# опишем функцию коррекции заполненных категориальных значений
def corr_cat(data, num_cat):
    ft_list=list(na_val(data).index)
    lst_count=pd.DataFrame(columns=[['feature_count_values']])
    for i in range (na_val(data).shape[0]):
        if len(list(data[ft_list[i]].value_counts()))<= num_cat: 
            lst_count.loc[ft_list[i]]=len(list(data[ft_list[i]].value_counts()))
    data[list(lst_count.index)].astype(float).round()
    return(data)

# модель добивания бедных классов 1
def model_bal_1(X_train, y_train):
    X_res, y_res = SMOTE().fit_resample(X_train, y_train)
    return(X_res,y_res)

# модель добивания бедных классов 2
def model_bal_2(X_train, y_train):
    X_res, y_res = ADASYN().fit_resample(X_train, y_train)
    return(X_res,y_res)
    
# модель добивания бедных классов 3
def model_bal_3(X_train, y_train):
    X_res, y_res = BorderlineSMOTE().fit_resample(X_train, y_train)
    return(X_res,y_res)

In [3]:
# читаем данные
train=pd.read_csv('contest_train.csv')
test=pd.read_csv('contest_test.csv')
# выводим важные параметры
print('Размер train:', train.shape)
print('Размер test:', test.shape)
print(train.TARGET.value_counts())
# удаляем нулевые/пустые столбцы
train=del_zero(train)
# удаляем все признаки в которых больше ХХХ пустых значений
train=del_na(train, 3000) # 3
# удаляем все признаки в которых есть дисбаланс, одно из значений сильно подавляющее
train=del_disbal(train, 18000) # 18


Размер train: (18390, 262)
Размер test: (6131, 261)
0    13029
1     4237
2     1124
Name: TARGET, dtype: int64
Пустые столбцы  ['FEATURE_3', 'FEATURE_144', 'FEATURE_249', 'FEATURE_256']  удалены
 Признаки с количеством порпусков > 3000 
             null_sum null_in_0 null_in_1 null_in_2
FEATURE_187     8857      6502      1849       506
FEATURE_189    18146     12854      4178      1114
FEATURE_190    12889      9201      2826       862
FEATURE_191    11451      8307      2423       721
FEATURE_192     9752      7040      2081       631
FEATURE_193     9382      6823      1977       582
FEATURE_194    12952      9081      2994       877  удалены
 Признаки с дисбалансом > 18000 
             max_sum max_in_0 max_in_1 max_in_2
FEATURE_5     18386    13026     4236     1124
FEATURE_6     18172    12877     4185     1110
FEATURE_20    18349    13003     4225     1121
FEATURE_27    18358    13004     4231     1123
FEATURE_28    18364    13010     4231     1123
FEATURE_29    18363    13013

In [4]:
# заполним порпущенные значения 
train=imputer(train,3)
# скорректируем заполненные категориальные значения
train=corr_cat(train, 7)

In [5]:
X=train.iloc[:,1:-1]
y=train.TARGET
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=0)
target_names = ['class 0', 'class 1', 'class 2']

In [6]:
X_test, y_test = model_bal_3(X_test, y_test)


In [7]:
clf = BernoulliNB(alpha = 1)
clf.fit(X_train, y_train)
pred=clf.predict(X_test)
print(classification_report(y_test, pred, target_names=target_names))
print('Fmacro мера:',f1_score(y_test, pred, average='macro'))

              precision    recall  f1-score   support

     class 0       0.54      0.58      0.56      3840
     class 1       0.51      0.42      0.46      3840
     class 2       0.66      0.72      0.69      3840

    accuracy                           0.58     11520
   macro avg       0.57      0.58      0.57     11520
weighted avg       0.57      0.58      0.57     11520

Fmacro мера: 0.5717210417485245
